In [1]:
import requests 
import json 
import csv
import pymongo
from pymongo import MongoClient
import time

In [2]:
client = MongoClient('localhost', 27017)
db = client['VKR']
collectionMongo = client['VKR1']['research_paper']
api_key='TMDfr4WmyPQiYpaZsCXAhcOzG1xlwqSg'
apikey = 'XUJ8Khm1sDBRovdxcG5fZ3STFEkjMOqt'
api_endpoint = "https://api.core.ac.uk/v3/"

In [3]:
def query_api(search_url, query, scrollId=None):
    headers={"Authorization":"Bearer "+ api_key}
    if scrollId:
        response = requests.get(f"{search_url}?q={query}&limit=300&scrollId={scrollId}",headers=headers)
    else:
        response = requests.get(f"{search_url}?q={query}&limit=400&scroll=true",headers=headers)
    if response.status_code ==200:
        return response.json(), response.elapsed.total_seconds()
    else:
        print(f"Error code {response.status_code}, {response.content}")
def scroll(search_url, query):
    count=0
    scrollId=None
    columns_to_delete = ['arxivId','dataProviders','fullText','id','identifiers','magId','oaiIds','pubmedId', 'links','outputs']
    while True:
        result, elapsed =query_api(search_url, query, scrollId)
        scrollId=result["scrollId"]
        totalhits = result["totalHits"]
        result_size = len(result["results"])
        if result_size==0:
            break
        for columns in columns_to_delete:
            for i in range(len(result["results"])):
                del result['results'][i][columns]
        collectionMongo.insert_many(result['results'])
        count+=result_size
        print(f"{count}/{totalhits} {elapsed}s  ")
    return print('It is done')

In [ ]:
query= f"big data analytics AND _exists_:abstract AND yearPublished:2011"
scroll("https://api.core.ac.uk/v3/search/works/",query)

388/1240 4.885319s  
774/1240 8.946947s  
1168/1240 21.340679s  
